In [2]:
# This if POC for testing LangChain implementation with Gymogul

In [3]:
# ChatOpenAI

In [9]:
from dotenv import dotenv_values
import openai
import re
import httpx
import os
from openai import OpenAI
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage

In [3]:
config = dotenv_values(".env")

client = OpenAI(api_key=config["OPEN_AI_KEY"])

In [4]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4.1-mini", model_provider="openai", api_key=config["OPEN_AI_KEY"])

In [46]:

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello, Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_c064fdde7c', 'finish_reason': 'stop', 'logprobs': None}, id='run--258fc923-09ed-4f07-bec0-9f8243200855-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [47]:
model.invoke("What's my name?")


AIMessage(content='I don’t know your name based on our current conversation. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 11, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_c064fdde7c', 'finish_reason': 'stop', 'logprobs': None}, id='run--52f909d5-1adc-433f-a020-0bb3f5eea596-0', usage_metadata={'input_tokens': 11, 'output_tokens': 19, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [48]:
model.invoke([HumanMessage(content="What's my name?")])


AIMessage(content="I don't have access to your personal information unless you share it with me. What would you like me to call you?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 11, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_c064fdde7c', 'finish_reason': 'stop', 'logprobs': None}, id='run--a5225aa9-f01e-4204-9dba-06ef98520ed6-0', usage_metadata={'input_tokens': 11, 'output_tokens': 24, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [52]:
# This is a SIMPLE example of how we can store chat history


In [49]:

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        # AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="Are you able to process image frames from a video?")
    ]
)

AIMessage(content="Hi Bob! I can analyze and interpret images you share with me, including individual frames extracted from a video. However, I don't have the capability to process video files directly or handle continuous video streams. If you extract specific frames from your video and upload them as images, I can certainly help analyze those! Let me know how you'd like to proceed.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 26, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_c064fdde7c', 'finish_reason': 'stop', 'logprobs': None}, id='run--717fb43c-f755-4b28-a423-7874a5310d35-0', usage_metadata={'input_tokens': 26, 'output_tokens': 70, 'total_tokens': 96, 'input_token_det

In [54]:
# This is an example of how we can process images and use in LangChain messages


In [5]:
def process_video(video_path, seconds_per_frame=2):
    base64Frames = []
    base_video_path, _ = os.path.splitext(video_path)

    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    frames_to_skip = int(fps * seconds_per_frame)
    curr_frame=0

    # Loop through the video and extract frames at specified sampling rate
    while curr_frame < total_frames - 1:
        video.set(cv2.CAP_PROP_POS_FRAMES, curr_frame)
        success, frame = video.read()
        if not success:
            break
        _, buffer = cv2.imencode(".jpg", frame)
        base64Frames.append(base64.b64encode(buffer).decode("utf-8"))
        curr_frame += frames_to_skip
    video.release()

    print(f"Extracted {len(base64Frames)} frames")
    return base64Frames



In [6]:
import cv2
import base64
video_path = "pushup.MOV"
base64Frames = process_video(video_path, seconds_per_frame=1)

Extracted 21 frames


In [10]:
messages = [
    SystemMessage(content="You are generating a video summary. Please provide a summary of the video. Respond in Markdown."),
    HumanMessage(
        content=[
            {"type": "text", "text": "Please analyze the frames from this video and tell me what is happening."},
            *[
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{frame}"},
                }
                for frame in base64Frames
            ]
        ]
    )
]

In [11]:
model.invoke(messages)

AIMessage(content="These frames depict a sequence where a person performs an exercise or workout routine involving a transition from standing to a crawling-like position and then moving into a plank or push-up position. Here's what is happening step-by-step:\n\n1. The person starts in a standing position with knees bent.\n2. They prepare for movement by slightly leaning forward.\n3. They bend down further, placing their hands closer to the floor.\n4. The person places their hands on the floor while maintaining a bent-knee position.\n5-11. The person shifts their weight forward, moving the body into a plank position, possibly transitioning between a crouch and a push-up/plank.\n12-14. The person appears to be in a full plank or push-up position.\n15. The person shifts back or finishes the plank position.\n16-17. The person moves back to a crouched position.\n18-19. The person stands back up and faces the camera with a slight smile.\n\nOverall, it looks like an exercise involving dynamic

In [ ]:
# We can try this approach to remove any heavy HumanMessage

In [45]:
messages = [
    SystemMessage(content="You are generating a video summary. Please provide a summary of the video. Respond in Markdown."),
    HumanMessage(
        content=[
            {"type": "text", "text": "Please analyze the frames from this video and tell me what is happening."},
            *[
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{frame}"},
                }
                for frame in base64Frames
            ]
        ]
    ),
    HumanMessage(content="My name is Allison. What is your name?")
]

In [49]:
# Removing the heavy image message
for msg in messages:
    if isinstance(msg.content, list):
        messages.remove(msg)